In [9]:
import xgboost as xgb

In [150]:
# import libraries
import pandas as pd
import numpy as np
from numpy import loadtxt
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler

In [199]:
# reading in the dataframe
calcofi_dat = pd.read_csv('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/calcofi_data/train.csv')

# removing the empty column from the dataframe
calcofi_dat = calcofi_dat.drop(['Unnamed: 12', 'id'], axis=1)
calcofi_dat.rename(columns = {"TA1.x": "TA1"}, inplace = True)
calcofi_dat

,Lat_Dec,Lon_Dec,NO2uM,NO3uM,NH3uM,R_TEMP,R_Depth,R_Sal,R_DYNHT,R_Nuts,R_Oxy_micromol.Kg,PO4uM,SiO3uM,TA1,Salinity1,Temperature_degC,DIC
0,34.385030,-120.665530,0.030,33.80,0.00,7.79,323,141.2,0.642,0.00,37.40948,2.77,53.86,2287.45,34.1980,7.82,2270.17
1,31.418333,-121.998333,0.000,34.70,0.00,7.12,323,140.8,0.767,0.00,64.81441,2.57,52.50,2279.10,34.0740,7.15,2254.10
2,34.385030,-120.665530,0.180,14.20,0.00,11.68,50,246.8,0.144,0.00,180.29150,1.29,13.01,2230.80,33.5370,11.68,2111.04
3,33.482580,-122.533070,0.013,29.67,0.01,8.33,232,158.5,0.562,0.01,89.62595,2.27,38.98,2265.85,34.0480,8.36,2223.41
4,31.414320,-121.997670,0.000,33.10,0.05,7.53,323,143.4,0.740,0.05,60.03062,2.53,49.28,2278.49,34.1170,7.57,2252.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,33.420000,-117.901667,0.000,38.60,0.00,6.52,520,118.2,0.889,0.00,12.17673,3.11,73.10,2313.52,34.3024,6.57,2311.19
1450,34.271667,-120.023333,0.330,5.90,0.00,13.21,49,290.1,0.149,0.00,224.81120,0.75,6.50,2223.31,33.3304,13.22,2048.93
1451,31.410000,-121.991667,0.000,33.60,0.00,7.61,322,143.1,0.731,0.00,60.46569,2.56,49.50,2280.43,34.1269,7.64,2251.34
1452,34.274270,-120.030570,0.517,1.58,0.35,14.47,30,319.0,0.110,0.35,249.28420,0.54,4.12,2223.88,33.2660,14.47,2030.03


In [209]:
# splitting the data into features and target
X = calcofi_dat.drop('DIC', axis = 1)
y = calcofi_dat['DIC']

# splitting in training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 36)

# creating a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)

In [152]:
# looking at what the X looks like
X

array([[ 1.25002502, -0.26125494, -0.11339488, ...,  0.89184386,
         1.08947294, -0.83647135],
       [-2.07977122, -1.03646427, -0.21887287, ...,  0.65464826,
         0.77812781, -1.01835386],
       [ 1.25002502, -0.26125494,  0.41399506, ..., -0.71739337,
        -0.57019749,  0.21138908],
       ...,
       [-2.08912411, -1.03258707, -0.21887287, ...,  0.69242912,
         0.91095166, -0.88533531],
       [ 1.12570891,  0.10806214,  1.59886446, ..., -0.91396745,
        -1.25063725,  0.96878043],
       [-0.95435238,  0.14702609, -0.19777727, ..., -0.52025116,
        -0.67314225,  1.61215588]])

In [314]:
# creating our no information MAE
mean_train = np.mean(y_train)

# get predictions from the test
baseline_predictions = np.ones(y_test.shape) * mean_train

# computing MAE
mse_baseline = metrics.mean_squared_error(y_test, baseline_predictions)

print("Baseline RMSE is {:.2f}".format(mse_baseline))

Baseline RMSE is 13468.60


In [319]:
# setting up the parameter that we will tune
params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': 0.3,
    'colsample_bytree': 1,
    'gamma': 0,
    'subsample': 0.8,
    'lambda': 0.5
    }

params['eval_metric'] = 'rmse'
num_boost_round = 999

In [316]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round = num_boost_round,
    evals = [(dtest, "Test")],
    early_stopping_rounds = 50
)

print("Best RMSE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

[0]	Test-rmse:1507.84757
[1]	Test-rmse:1057.92380
[2]	Test-rmse:741.60629
[3]	Test-rmse:520.52880
[4]	Test-rmse:365.63466
[5]	Test-rmse:256.70149
[6]	Test-rmse:180.48796
[7]	Test-rmse:127.20329
[8]	Test-rmse:89.93422
[9]	Test-rmse:63.98475
[10]	Test-rmse:45.86452
[11]	Test-rmse:33.14143
[12]	Test-rmse:24.34760
[13]	Test-rmse:18.29815
[14]	Test-rmse:14.37090
[15]	Test-rmse:11.75078
[16]	Test-rmse:10.04790
[17]	Test-rmse:9.13341
[18]	Test-rmse:8.40037
[19]	Test-rmse:7.98317
[20]	Test-rmse:7.79951
[21]	Test-rmse:7.61166
[22]	Test-rmse:7.55203
[23]	Test-rmse:7.47965
[24]	Test-rmse:7.37660
[25]	Test-rmse:7.34288
[26]	Test-rmse:7.28759
[27]	Test-rmse:7.29716
[28]	Test-rmse:7.30274
[29]	Test-rmse:7.22799
[30]	Test-rmse:7.23478
[31]	Test-rmse:7.18124
[32]	Test-rmse:7.16437
[33]	Test-rmse:7.12696
[34]	Test-rmse:7.10973
[35]	Test-rmse:7.07268
[36]	Test-rmse:7.05558
[37]	Test-rmse:7.03483
[38]	Test-rmse:6.98280
[39]	Test-rmse:6.98899
[40]	Test-rmse:6.96717
[41]	Test-rmse:6.94628
[42]	Test-rmse:6.

In [317]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round = num_boost_round,
    seed = 36,
    metrics = {'rmse'},
    early_stopping_rounds = 50
)

cv_results

cv_results['test-rmse-mean'].min()

7.3667057560542055

In [318]:
# setting up the parameters to add constraints on the architecture of the trees
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3,12, 1)
    for min_child_weight in range(1,6, 1)
]

In [301]:
# defining the best params and MAE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_dept = {}, min_child_weight = {}". format(max_depth, min_child_weight))

    # updating the parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    # Run cross validation
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round = num_boost_round,
        seed = 36,
        nfold = 10,
        metrics = {'rmse'},
        early_stopping_rounds = 50
    )

    # update best MAE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    #print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse= mean_rmse
        best_params = (max_depth, min_child_weight)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


CV with max_dept = 3, min_child_weight = 1
[17:16:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:16:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:16:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:16:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:16:15] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpyth

In [ ]:
# defining the best params and MAE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_dept = {}, min_child_weight = {}". format(max_depth, min_child_weight))

    # updating the parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight

    # Run cross validation
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round = num_boost_round,
        seed = 36,
        nfold = 10,
        metrics = {'rmse'},
        early_stopping_rounds = 50
    )

    # update best MAE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    #print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse= mean_rmse
        best_params = (max_depth, min_child_weight)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))


In [302]:
params['max_depth'] = 3
params['min_child_weight'] = 2

In [303]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range (7,11)]
    for colsample in [i/10. for i in range(7, 11)]
]

In [304]:
min_rmse = float("Inf")
best_params = None

# start by the largest values and go down to the smallest value
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample ={}, colsample ={}".format(subsample, colsample))

    # update the parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample

    # running cross validation on this information
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round = num_boost_round,
        seed = 36,
        nfold = 10,
        metrics = {'rmse'},
        early_stopping_rounds = 50
    )

    # updating best score for evaluating
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_rmse, boost_rounds))

    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample, colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

CV with subsample =1.0, colsample =1.0
[17:18:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:18:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:18:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:18:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:18:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-3

In [305]:
# updating our parameter dictionary
params['subsample'] = 1.0
params['colsample_bytree'] = 0.8

In [306]:
%time

# fitting the ETA (learning rate) parameter
min_rmse = float("Inf")
best_params = None

for eta in [.5, .4, .3, .2, .1, 0.05, .025, 0.01, 0.005]:
    print("CV with eta = {}".format(eta))

    # update the parameter
    params['eta'] = eta

    # running this and adding the CV (adding time to see improvement with computational time)
    %time cv_results = xgb.cv(params, dtrain, num_boost_round = num_boost_round, seed = 36, nfold = 5, metrics = ['rmse'], early_stopping_rounds = 10)

    # update the best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds\n".format(mean_rmse, boost_rounds))

    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta

print("Best params: {}, RMSE: {}".format(best_params, min_rmse))



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
CV with eta = 0.5
[17:21:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:21:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:21:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:21:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[17:21:10] WARNING: /Users/runner/work/xgboost/xgboost/python-pack

In [307]:
# saving eta parameter
params['eta'] = 0.01

In [308]:
# looking at the parameter dictionary
params

{'max_depth': 3,
 'min_child_weight': 2,
 'eta': 0.01,
 'colsample_bytree': 0.8,
 'model__n_estimators': 50,
 'model__subsample': 0.8,
 'eval_metric': 'rmse',
 'subsample': 1.0}

In [309]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round = num_boost_round,
    evals = [(dtest, "Test")],
    early_stopping_rounds = 50
)

print("Best RMSE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

[17:22:48] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "model__n_estimators", "model__subsample" } are not used.

[0]	Test-rmse:2130.09503
[1]	Test-rmse:2108.88293
[2]	Test-rmse:2087.87147
[3]	Test-rmse:2067.06933
[4]	Test-rmse:2046.48576
[5]	Test-rmse:2026.09805
[6]	Test-rmse:2005.92435
[7]	Test-rmse:1985.95327
[8]	Test-rmse:1966.16853
[9]	Test-rmse:1946.58250
[10]	Test-rmse:1927.20381
[11]	Test-rmse:1908.01351
[12]	Test-rmse:1889.01937
[13]	Test-rmse:1870.20221
[14]	Test-rmse:1851.57703
[15]	Test-rmse:1833.13325
[16]	Test-rmse:1814.89583
[17]	Test-rmse:1796.82089
[18]	Test-rmse:1778.93604
[19]	Test-rmse:1761.21790
[20]	Test-rmse:1743.69185
[21]	Test-rmse:1726.31239
[22]	Test-rmse:1709.15002
[23]	Test-rmse:1692.09699
[24]	Test-rmse:1675.27579
[25]	Test-rmse:1658.56776
[26]	Test-rmse:1642.03021
[27]	Test-rmse:1625.70544
[28]	Test-rmse:1609.48740
[29]	Test-rmse:1593.49540
[30]	Test-rmse:15

In [321]:
# saving the number of boosted rounds
num_boost_round = model.best_iteration

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round = num_boost_round,
    evals = [(dtest, "Test")]
)

[0]	Test-rmse:1507.84757
[1]	Test-rmse:1057.92380
[2]	Test-rmse:741.60629
[3]	Test-rmse:520.52880
[4]	Test-rmse:365.63466
[5]	Test-rmse:256.70149
[6]	Test-rmse:180.48796
[7]	Test-rmse:127.20329
[8]	Test-rmse:89.93422
[9]	Test-rmse:63.98475
[10]	Test-rmse:45.86452
[11]	Test-rmse:33.14143
[12]	Test-rmse:24.34760
[13]	Test-rmse:18.29815
[14]	Test-rmse:14.37090
[15]	Test-rmse:11.75078
[16]	Test-rmse:10.04790
[17]	Test-rmse:9.13341
[18]	Test-rmse:8.40037
[19]	Test-rmse:7.98317
[20]	Test-rmse:7.79951
[21]	Test-rmse:7.61166
[22]	Test-rmse:7.55203
[23]	Test-rmse:7.47965
[24]	Test-rmse:7.37660
[25]	Test-rmse:7.34288
[26]	Test-rmse:7.28759
[27]	Test-rmse:7.29716
[28]	Test-rmse:7.30274
[29]	Test-rmse:7.22799
[30]	Test-rmse:7.23478
[31]	Test-rmse:7.18124
[32]	Test-rmse:7.16437
[33]	Test-rmse:7.12696
[34]	Test-rmse:7.10973
[35]	Test-rmse:7.07268
[36]	Test-rmse:7.05558
[37]	Test-rmse:7.03483
[38]	Test-rmse:6.98280
[39]	Test-rmse:6.98899
[40]	Test-rmse:6.96717
[41]	Test-rmse:6.94628
[42]	Test-rmse:6.

In [322]:
# using the model to make predictions
metrics.mean_squared_error(best_model.predict(dtest), y_test)

46.326103313692876

In [323]:
# saving the model to use on new data
best_model.save_model("calcofi_model")

In [324]:
# loading in the testing data
calcofi_pred_dat = pd.read_csv('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/calcofi_data/test.csv')
# creating an ID Column to combine with the predictions
id = calcofi_pred_dat['id']

# dropping the ID column to match the training data
calcofi_pred_dat = calcofi_pred_dat.drop(columns= ['id'], axis = 1)

# turning prediction data to a DMatrix for running the model
dpredict = xgb.DMatrix(calcofi_pred_dat)

# predicting on new data
new_predictions = best_model.predict(dpredict)

# turning new predictions to a dataframe
dic_predictions = pd.DataFrame(new_predictions)

# renaming the column names
dic_predictions.rename(columns = {0 : 'DIC'}, inplace = True)

# adding the ID column
dic_predictions = pd.concat([dic_predictions, id], axis = 1)

# chaning the order of the columns
dic_predictions = dic_predictions[['id', 'DIC']]

#dic_predictions.rename(columns = {'id' : 'test.id'}, inplace = True)
dic_predictions

,id,DIC
0,1455,2169.831299
1,1456,2194.771729
2,1457,2317.135010
3,1458,1989.241699
4,1459,2152.014160
...,...,...
480,1935,2002.496094
481,1936,2177.996582
482,1937,2244.769043
483,1938,2015.266479


In [325]:
from pathlib import Path

# writing the predictions as a csv
filepath = Path('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/predictions.csv')
dic_predictions.to_csv(filepath, index = False)

############# STOP HERE

In [204]:
# loading the model with the model
loaded_model = xgb.Booster()
loaded_model.load_model("calcofi_model")

# using the model for predictions
calcofi_pred_dat = pd.read_csv('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/calcofi_data/test.csv')

# creating a DMatrix for XGBoost
dpredict = xgb.DMatrix(calcofi_pred_dat)
dpredict
#loaded_model.predict(dpredict)

105.10967592829908

In [126]:
# reading in test data
calcofi_pred_dat = pd.read_csv('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/calcofi_data/test.csv')
#calcofi_pred_dat
# predicting on new data
new_predictions = xgb_model_best.predict(calcofi_pred_dat)



In [133]:
id = calcofi_pred_dat['id']

0      1455
1      1456
2      1457
3      1458
4      1459
       ... 
480    1935
481    1936
482    1937
483    1938
484    1939
Name: id, Length: 485, dtype: int64

In [140]:
dic_predictions = pd.DataFrame(new_predictions) # creating a dataframe of the new predictions
dic_predictions.rename(columns = {0 : 'DIC'}, inplace = True) #
dic_predictions = pd.concat([dic_predictions, id], axis = 1) # adding the ID column
dic_predictions = dic_predictions[['id', 'DIC']] # chaning the order of the columns
#dic_predictions.rename(columns = {'id' : 'test.id'}, inplace = True)
dic_predictions

,id,DIC
0,1455,2167.864990
1,1456,2203.789795
2,1457,2318.203369
3,1458,1992.757935
4,1459,2146.689697
...,...,...
480,1935,2000.060669
481,1936,2189.401123
482,1937,2246.179688
483,1938,2021.887085


In [141]:
from pathlib import Path
filepath = Path('/Users/colleenmccamy/Documents/MEDS/classes/winter/eds_232_machine_learning/labs/predictions.csv')
dic_predictions.to_csv(filepath, index = False)